<a href="https://colab.research.google.com/github/yiyc-kr/llm-study/blob/main/ko_en_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
drive_path = 'drive/MyDrive/llm-study/'

In [4]:
# GPU 사용 체크
# TPU를 사용하는 경우 !nvidia-smi 명령어는 작동하지 않습니다.
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 27 10:26:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0              47W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [5]:
# 고용량 메모리 사용 체크
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


데이터 다운로드

In [6]:
# !pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [7]:
import datasets
datasets.__version__ # 2.19.1

'2.19.1'

In [8]:
from datasets import load_dataset

In [9]:
en_ko = load_dataset("bongsoo/news_talk_en_ko")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1299999 [00:00<?, ? examples/s]

In [10]:
en_ko

DatasetDict({
    train: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 1299999
    })
})

In [11]:
en_ko['train']

Dataset({
    features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
    num_rows: 1299999
})

In [12]:
import pandas as pd

In [13]:
en_ko_df = pd.DataFrame(en_ko['train'])
en_ko_df.head()

,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
0,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
1,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
2,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
3,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.
4,I can't even consider uninsured treatments.,보험 처리가 안 되는 비급여 시술은 엄두도 못 낸다.


In [14]:
del en_ko

In [15]:
column_names = ["en", "ko"]
df_forward = pd.DataFrame([en_ko_df.columns], columns=column_names)
en_ko_df.columns = column_names
en_ko_df = pd.concat([df_forward, en_ko_df]).reset_index(drop=True)
en_ko_df.head()

,en,ko
0,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
1,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
2,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
3,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
4,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.


In [16]:
del df_forward

In [17]:
len(en_ko_df)

1300000

In [18]:
from datasets import Dataset
dataset = Dataset.from_pandas(en_ko_df)

In [19]:
dataset

Dataset({
    features: ['en', 'ko'],
    num_rows: 1300000
})

In [20]:
# 각 데이터 셋의 샘플 수를 정한다.
num_train = 1200000
num_valid = 90000
num_test = 10000

In [21]:
en_ko_df_train = en_ko_df.iloc[:num_train]
en_ko_df_valid = en_ko_df.iloc[num_train:num_train+num_valid]
en_ko_df_test = en_ko_df.iloc[-num_test:]

In [25]:
en_ko_df_train.to_csv(drive_path+"train.tsv", sep='\t', index=False)
en_ko_df_valid.to_csv(drive_path+"valid.tsv", sep='\t', index=False)
en_ko_df_test.to_csv(drive_path+"test.tsv", sep='\t', index=False)

In [26]:
data_files = {"train": drive_path+"train.tsv", "valid": drive_path+"valid.tsv", "test": drive_path+"test.tsv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 1200000
    })
    valid: Dataset({
        features: ['en', 'ko'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['en', 'ko'],
        num_rows: 10000
    })
})

In [28]:
dataset['train'][10]['en'], dataset['train'][10]['ko']

('Any academic achievement requires constant repetition.',
 '어떤 학문이든지 일정의 성취를 이루기 위해서는 끊임없는 반복이 필요하다.')

In [29]:
del en_ko_df_train
del en_ko_df_valid
del en_ko_df_test
del en_ko_df

데이터 토큰화

In [30]:
import torch
torch.__version__ # 2.3.0+cu121

'2.3.0+cu121'

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [32]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

In [33]:
from transformers import AutoTokenizer

In [34]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [35]:
def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['ko'],
                             text_target=examples['en'],
                             max_length=max_token_length, truncation=True)

    return model_inputs

In [36]:
import multiprocessing

In [37]:
NUM_CPU = multiprocessing.cpu_count()
NUM_CPU

12

In [38]:
tokenized_datasets = dataset.map(convert_examples_to_features,
                                 batched=True,
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서
                                 # 아래서 콜레이터가 패딩을 못해서 에러남
                                 remove_columns=dataset["train"].column_names,
                                 num_proc=NUM_CPU)

Map (num_proc=12):   0%|          | 0/1200000 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/90000 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [39]:
from transformers import AutoModelForSeq2SeqLM

In [ ]:
# model_ckpt = "KETI-AIR/ke-t5-base"
# device = "cuda"
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
evaluate.__version__ # 0.4.2

In [ ]:
!pip install sacrebleu

In [ ]:
import sacrebleu
sacrebleu.__version__ # 2.4.2

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np
np.__version__ # 1.25.2

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]/100}

    return result

In [ ]:
from transformers import EarlyStoppingCallback, Seq2SeqTrainingArguments

In [ ]:
import transformers
transformers.__version__ # 4.41.0

In [ ]:
!pip install accelerate

In [ ]:
import accelerate
accelerate.__version__

In [ ]:
!pip install git+https://github.com/huggingface/accelerate # 이게 해결법이 되엇다!

In [ ]:
# 새로운 TrainingArguments 설정
training_args = Seq2SeqTrainingArguments(
    output_dir=drive_path+"ko_en_chkpt",
    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    num_train_epochs=1,  # 에폭 수 조절
    save_steps=500,
    save_total_limit=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

In [ ]:
# 기존 체크포인트 경로
# checkpoint = "chkpt/checkpoint-9000"

# 새로운 Trainer 설정
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # 조기 종료 추가
)

# 기존 체크포인트에서 학습 재개
# trainer.train(resume_from_checkpoint=checkpoint)

# 그냥 새로 학습
trainer.train()

In [ ]:
# !pip install transformers[torch]

In [ ]:
# !pip install --upgrade transformers[torch]
# !pip install --upgrade accelerate

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,No log,2.698199,6.367352


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=9375, training_loss=3.065775625, metrics={'train_runtime': 8585.2298, 'train_samples_per_second': 139.775, 'train_steps_per_second': 1.092, 'total_flos': 9.074606753415168e+16, 'train_loss': 3.065775625, 'epoch': 1.0})

In [ ]:
trainer.save_model("./results")

In [ ]:
!tar -czvf results.tar.gz results/

results/
results/tokenizer.json
results/spiece.model
results/model.safetensors
results/tokenizer_config.json
results/config.json
results/generation_config.json
results/special_tokens_map.json
results/training_args.bin


In [ ]:
model_dir = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu()

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


T5ForConditionalGeneration(
  (shared): Embedding(64128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
input_text = [
    "I got my peaches out in Georgia (oh, yeah, shit)",
    "I'm hungry. But I won't eat. Because I'm on a diet. I have to exercise, but it's annoying. I just want to sleep."
]

In [ ]:
inputs = tokenizer(input_text, return_tensors="pt",
                   padding=True, max_length=max_token_length)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
koreans = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

In [ ]:
[
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(korean)) for korean in koreans
]

['<pad> 나는 조지아에서 복숭아를 땄어요(오, 맞아요, <unk>).</s><pad><pad><pad><pad><pad><pad>',
 '<pad> 저는 배가 <unk>지만 먹지 않을 것이에요, 다이어트가 돼서 운동해야 하는데 귀찮고, 그냥 자고 싶어.</s>']

In [ ]:
new_trainer.save_model("./new_results")